In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore


from langchain_community.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

In [88]:
def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []

In [89]:
import re
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

from langchain.document_loaders import PyMuPDFLoader
def load_and_split_tax_law(file_name):
    
    """
    PDF 파일들을 처리하여 임베딩을 Chroma Vector Store에 저장합니다.
    """

    file_path = f"data/tax_law/{file_name}.pdf"

    loader = PyMuPDFLoader(file_path)
    load_document = loader.load()

    # 전처리 - 반복 텍스트 삭제
    delete_pattern_1 = rf"법제처\s*\d+\s*국가법령정보센터\n{file_name.replace('_', ' ')}\n" 
    delete_pattern_2 = r'\[[\s\S]*?\]'
    delete_pattern_3 = r'<[\s\S]*?>'
    
    full_text = " ".join([document.page_content for document in load_document])
    
    full_text = re.sub(delete_pattern_1, "", full_text)
    full_text=re.sub(delete_pattern_2, '', full_text)
    full_text=re.sub(delete_pattern_3, '', full_text)
    
    # 전처리 - split
    split_pattern = r"\s*\n(제\d+조(?:의\d+)?(?:\([^)]*\))?)(?=\s|$)"
    chunks = re.split(split_pattern, full_text)

    chunk_docs = []  # 최종 return 할 list
    connected_chunks = [] 
    current_chunk = ""
    is_buchik_section = False 
    
    # 전처리 - 일반 조항과 부칙의 조항과 구별하기 위해 접두어 '부칙-'을 넣음.
    for chunk in chunks:
        if re.search(r"\n\s*부칙", chunk): 
            is_buchik_section = True

        if chunk.startswith("제") and "조" in chunk: 
            if is_buchik_section:
                chunk = "부칙-" + chunk

            if current_chunk:
                connected_chunks.append(current_chunk.strip())
            current_chunk = chunk 
        else:
            current_chunk += f" {chunk}" 

    if current_chunk:
        connected_chunks.append(current_chunk.strip())

    for chunk in connected_chunks:
        pattern =  r"^(?:부칙-)?제\d+조(?:의\d*)?(?:\([^)]*\))?"

        keyword = []

        keyword.append(file_name)
        
        match = re.search(pattern, chunk)
        if match:
            word = match.group()
            word = re.sub(r'\s+', ' ', word)
            keyword.append(word) 
            
        doc = Document(metadata={"title": file_name, "keyword":keyword, "effective_year": 2025 }, page_content=chunk),
        
        chunk_docs.extend(doc)
        
    return chunk_docs


In [95]:

# print(len(all_docs))
# print(all_docs[1])
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# vector_store = Chroma.from_documents(
#     documents=all_docs,
#     embedding=embedding_model,
#     collection_name=COLLECTION_NAME,
#     persist_directory=PERSIST_DIRECTORY
# )



all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_and_split_tax_law(file))
# load - 세법

5431

In [48]:
# Prompt Template 생성
messages = [
        ("ai", """
        당신은 대한민국 세법에 대해 전문적으로 학습된 AI 도우미입니다. 저장된 세법 조항 데이터를 기반으로 사용자 질문에 답변하세요.

        - 모든 답변은 학습된 세법 데이터 내에서만 유효한 정보를 바탕으로 작성하세요. 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.
        - 질문에 명확한 답변이 없거나 데이터 내에서 찾을 수 없는 경우, 정직하게 "잘 모르겠습니다."라고 말하고, 새로운 질문을 유도하세요.
        - 질문이 포함된 조항뿐 아니라, 필요 시 서로 연관된 다른 조항도 참고하여 답변의 정확성과 완성도를 높이세요.
        - 사용자가 이해하기 쉽게 답변을 구성하며, 중요한 키워드나 법 조항은 명확히 표시하세요.
        - 세법과 관련된 복잡한 질문에 대해서는 관련 조항 번호와 요약된 내용을 포함하여 답변을 제공하세요.
        
        추가 규칙:
        답변은 간결하고 명료하게 작성하되, 필요한 경우 관련 조항의 전문을 추가적으로 인용하세요.
        세법 용어를 사용자 친화적으로 설명하여 비전문가도 쉽게 이해할 수 있도록 하세요.
        질문을 완전히 이해하기 어렵거나 모호할 경우, 사용자가 구체적으로 질문을 다시 작성할 수 있도록 유도하는 후속 질문을 하세요.

	답변 후, 사용자에게 필요할 것 같은 정보를 바탕으로 두 가지 후속 질문을 제안하세요. 각 질문의 앞뒤에 한 줄씩 띄어쓰기를 하세요. 이 질문은 원래 주제와 관련된 내용이어야 합니다.
	특정 법률 조항이나 제도가 언급될 경우, 근거가 되는 세법 조문, 시행령, 또는 관련 자료를 명시합니다.
        모든 답변은 사용자에게 법적 조언이 아닌 정보 제공 목적으로 작성된 것임을 명확히 합니다. 
	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser


In [ ]:
# chain.invoke("개별소비세법이 뭐야?")
# chain.invoke("개별소비세법이 제1조가 뭐야")
# chain.invoke("교통_에너지_환경세법 뭐야?")
chain.invoke("조세범 처벌절차법의 정의해줘")
